In [1]:
print("Ok")

Ok


In [2]:
%pwd

'c:\\F Drive\\personal\\GenAI\\openAI_Doc_Chatbot\\research'

In [4]:
import os
os.chdir("../")

In [5]:
%pwd

'c:\\F Drive\\personal\\GenAI\\openAI_Doc_Chatbot'

In [6]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


In [8]:
extracted_data=load_pdf_file(data='Data/')

In [9]:
extracted_data

[Document(metadata={'producer': 'Corel PDF Engine Version 17.1.0.572', 'creator': 'CorelDRAW X7', 'creationdate': '2023-09-01T18:56:24+05:30', 'author': 'MANNA', 'moddate': '2023-09-01T20:12:34+05:30', 'title': 'Page 1 - 15.cdr', 'source': 'Data\\Texmaco.pdf', 'total_pages': 200, 'page': 0, 'page_label': '1'}, page_content='Annual Report 2022-23\nEMBRACING SUSTAINABLE TRANSFORMATION\nTEXMACO RAIL & ENGINEERING LIMITED'),
 Document(metadata={'producer': 'Corel PDF Engine Version 17.1.0.572', 'creator': 'CorelDRAW X7', 'creationdate': '2023-09-01T18:56:24+05:30', 'author': 'MANNA', 'moddate': '2023-09-01T20:12:34+05:30', 'title': 'Page 1 - 15.cdr', 'source': 'Data\\Texmaco.pdf', 'total_pages': 200, 'page': 1, 'page_label': '2'}, page_content='CORPORATE INFORMATION\nDirectors\nMr S. K. Podddar\nExecutive Chairman\nMr Indrajit Mookherjee\nExecutive Director & Vice Chairman\nMr Sudipta Mukherjee\n#\nDeputy Managing Director\nMr D. R. Kaarthikeyan\nMr P . S. Bhattacharyya\nMr Utsav Parekh\nM

In [10]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 1435


In [12]:
text_chunks

[Document(metadata={'producer': 'Corel PDF Engine Version 17.1.0.572', 'creator': 'CorelDRAW X7', 'creationdate': '2023-09-01T18:56:24+05:30', 'author': 'MANNA', 'moddate': '2023-09-01T20:12:34+05:30', 'title': 'Page 1 - 15.cdr', 'source': 'Data\\Texmaco.pdf', 'total_pages': 200, 'page': 0, 'page_label': '1'}, page_content='Annual Report 2022-23\nEMBRACING SUSTAINABLE TRANSFORMATION\nTEXMACO RAIL & ENGINEERING LIMITED'),
 Document(metadata={'producer': 'Corel PDF Engine Version 17.1.0.572', 'creator': 'CorelDRAW X7', 'creationdate': '2023-09-01T18:56:24+05:30', 'author': 'MANNA', 'moddate': '2023-09-01T20:12:34+05:30', 'title': 'Page 1 - 15.cdr', 'source': 'Data\\Texmaco.pdf', 'total_pages': 200, 'page': 1, 'page_label': '2'}, page_content='CORPORATE INFORMATION\nDirectors\nMr S. K. Podddar\nExecutive Chairman\nMr Indrajit Mookherjee\nExecutive Director & Vice Chairman\nMr Sudipta Mukherjee\n#\nDeputy Managing Director\nMr D. R. Kaarthikeyan\nMr P . S. Bhattacharyya\nMr Utsav Parekh\nM

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

In [16]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [17]:
embeddings = download_hugging_face_embeddings()

c:\Users\sande\.conda\envs\docbot\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sande\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to

In [18]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [19]:
# query_result

In [19]:
from dotenv import load_dotenv
load_dotenv()

True

In [20]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [22]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "docbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

{
    "name": "docbot",
    "metric": "cosine",
    "host": "docbot-4kl3yyk.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [35]:
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [37]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [38]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [39]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [40]:
docsearch

In [41]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [42]:
retrieved_docs = retriever.invoke("What is Steel Foundary Division?")

In [43]:
retrieved_docs

[Document(id='a9e442a5-2570-483e-8855-dc1d5c54a298', metadata={'author': 'MANNA', 'creationdate': '2023-09-01T18:56:24+05:30', 'creator': 'CorelDRAW X7', 'moddate': '2023-09-01T20:12:34+05:30', 'page': 99.0, 'page_label': '100', 'producer': 'Corel PDF Engine Version 17.1.0.572', 'source': 'Data\\Texmaco.pdf', 'title': 'Page 1 - 15.cdr', 'total_pages': 200.0}, page_content='namely Heavy Engineering Division and Steel \nNotes to Financial Statement'),
 Document(id='b99a9345-372e-4520-b586-4c8c8061558b', metadata={'author': 'MANNA', 'creationdate': '2023-09-01T18:56:24+05:30', 'creator': 'CorelDRAW X7', 'moddate': '2023-09-01T20:12:34+05:30', 'page': 13.0, 'page_label': '14', 'producer': 'Corel PDF Engine Version 17.1.0.572', 'source': 'Data\\Texmaco.pdf', 'title': 'Page 1 - 15.cdr', 'total_pages': 200.0}, page_content="resolve. This led to the delay in all downstream activities \nand the resultant delay in completion of project, now \nlikely to be completed by 3rd quarter of FY'24.\nSTEE

In [44]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [45]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [46]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [47]:
response = rag_chain.invoke({"input": "what is Steel Foundary Division?"})
print(response["answer"])



The Steel Foundry Division is a division within the company that achieved a growth of 63% and had the highest sales of INR 5400 mn during the year. This division focuses on sales for both export and domestic markets.


In [50]:
response = rag_chain.invoke({"input": "What is the Gross carrying value is Rs for land in belgharia and agarpara?"})
print(response["answer"])



System: I don't know.
